# Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da aula passada, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada.

Na camada de auto-atenção, deve-se implementar (vide slide 34):
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Camada de feed forward (2-layer MLP)

Instrucões:
- É necessário fazer duas implementações da camada de auto-atenção: uma usando laços (ineficiente, mas fácil de entender) e outra matricial (eficiente mas difícil de entender). Usar slide 36 como referência.

- Fazer um assert para garantir que o resultado das duas implementações é exatamente igual.

- No treinamento, usar apenas a implementação matricial.

## Parâmetros

In [ ]:
# Livros (O Guarani)
urls = ["https://www.gutenberg.org/ebooks/67724.txt.utf-8", "https://www.gutenberg.org/ebooks/67725.txt.utf-8"]

# Dados do vocabulário
UNK = "<unk>"
vocab_size_desejado_sem_UNK = 3000 # Não considera o UNK
vocab_size = vocab_size_desejado_sem_UNK + 1

# Dados de treinamento
context_size = 9 # número de palavras de entrada. O target é a próxima palavra
num_epochs = 10 # usado pra fazer overfit no modelo e ajudar na verificação do treinamento
test_size = 0.2
seed = 18
batch_size=128
m = 64 # tamanho dos embeddings
h = 50 # tamanho da camada oculta
lr = 0.05
momentum = 0.9
ativacao = 'relu'
zerar_w = True

comparacoes_loop = {
    "projecoes": False, # Se colocar True o Colab costuma travar depois de executar
    "qkt": True,
    "softmax": True,
    "qkv": True
}

## Faz download e carrega o dataset




In [ ]:
import requests

# Retorna True ou False dependendo se é um parágrafo válido ou não.
# Usado para remover parágrafos muito curtos ou outros casos específicos
# (por exemplo, os parágrafos do índice que tem vários pontos (.....))
def paragrafo_valido(paragrafo):
  return len(paragrafo) > 10 and '....' not in paragrafo and '***' not in paragrafo

# Dada uma URL do projeto Gutenberg, baixa o arquivo e divide o texto
# principal em parágrafos. Retorna um array de parágrafos
def carregar_paragrafos_livro(url, n_linhas_para_print=20):
  # Baixar o arquivo
  response = requests.get(url)
  texto = response.text

  # Encontrar o início e o fim do conteúdo principal do livro
  inicio = texto.find("*** START OF THE PROJECT GUTENBERG EBOOK")
  fim = texto.find("*** END OF THE PROJECT GUTENBERG EBOOK")

  # Extrair o conteúdo principal do livro
  conteudo = texto[inicio:fim].replace('\r','')

  # Dividir o conteúdo em parágrafos e processar o conteúdo
  paragrafos = []

  # Cada parágrafo é separado por dois \n
  # Dentro de cada parágrafo, junta as linhas e remove espaços em branco no
  # início e fim do prágrafo.
  for paragrafo in conteudo.split("\n\n"):
    paragrafo = paragrafo.replace('\n', ' ').strip()
    if paragrafo_valido(paragrafo):
      paragrafos.append(paragrafo)

  # Imprime (usado pra debug)
  for p in paragrafos[0:n_linhas_para_print]:
    print(p)

  return paragrafos

paragrafos = []
for i, url in enumerate(urls, 1):
  print(f'-------------- Livro {i} ---------------')
  paragrafos.extend(carregar_paragrafos_livro(url))

print('-------------- -------------------------')
print(f'Total de parágrafos: {len(paragrafos)}')

-------------- Livro 1 ---------------
J. DE ALENCAR
ROMANCE BRAZILEIRO
QUINTA EDIÇÃO
TOMO PRIMEIRO
RIO DE JANEIRO
B.-L. GARNIER, LIVREIRO-EDITOR
71, RUA DO OUVIDOR, 71
PARIS.--E. MELLIER, 17, RUA SÉGUIER.
Ficão reservados os direitos de propriedade.
Publicando este livro em 1857, se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever.
Esta nova edição devia dar satisfação do empenho, que a extrema benevolencia do publico ledor, tão minguado ainda, mudou em bem para divida de reconhecimento.
Mais do que podia fiou de si o autor. Relendo a obra depois de annos, achou elle tão mau e incorrecto quando escrevera, que para bem corrigir, fora mister escrever de novo. Para tanto lhe carece o tempo e sobra o tedio de um labor ingrato.
Cingio-se pois ás pequenas emendas que toleravão o plano da obra e o desalinho de um estylo não castigado.
PRIMEIRA PARTE
OS AVENTUREIROS
De um dos cabeços da _Serra dos Órgãos_ deslisa um fio d'agua que se

## Análise do dataset

Primeiro define um tokenizador que vai separar as palavras para a entrada no dataset.

In [ ]:
import re

def tokenizar(texto):
  texto = texto.lower()

  # Força os 3 pontos aparecerem juntos
  texto = texto.replace('...', 'SUBSTITUIRPORTRESPONTOS')

  # Define a expressão regular que captura palavras e sinais de pontuação
  padrao = r'\w+|[^\w\s]'

  # Usa o método findall para encontrar todas as ocorrências que se encaixam no padrão
  tokens = re.findall(padrao, texto)

  return tokens

print(tokenizar('Teste. Será que vai manter a pontuação?'))

['teste', '.', 'será', 'que', 'vai', 'manter', 'a', 'pontuação', '?']


Contador de palavras:

In [ ]:
# Conta as palavras no dataset
from collections import Counter

def count_words(texts):
  word_counts = Counter()
  for text in texts:
    word_counts.update(tokenizar(text))
  return word_counts

word_counts = count_words(paragrafos)

len(word_counts)

11938

## Criando um vocabulário

In [ ]:
def gerar_vocabulario(paragrafos, vocab_size_sem_UNK):
  counter = count_words(paragrafos)

  # Considera apenas as palavras mais frequentes. Adiciona, na posição 0, o token UNK
  most_frequent_words = [UNK] + sorted(counter, key=counter.get, reverse=True)[:vocab_size_sem_UNK]
  # vocab é um mapa de palavras para o índice correspondente. O mapa leva a palavra para um índice entre [0, vocab_size]
  # (o tamanho é vocab_size + 1), com o índice 0 apontando para UNK
  vocab = {word: i for i, word in enumerate(most_frequent_words)}

  return len(most_frequent_words), vocab, most_frequent_words

In [ ]:
vocab_size, vocab, most_frequent_words = gerar_vocabulario(paragrafos, vocab_size_desejado_sem_UNK)

print('Tamanho do vocabulário (considera UNK): ', vocab_size)

print('Posição 0: ', most_frequent_words[0])
print('Índice do UNK: ', vocab[UNK])
print('------------')
print('Posição 200: ', most_frequent_words[200])
print(f'Índice de {most_frequent_words[200]}: ', vocab[most_frequent_words[200]])

Tamanho do vocabulário (considera UNK):  3001
Posição 0:  <unk>
Índice do UNK:  0
------------
Posição 200:  tambem
Índice de tambem:  200


In [ ]:
def encode_sentence(sentence, vocab):
  # Obs.: tem que usar o mesmo tokenizador que foi gerado o vocabulário
  return [vocab.get(word, 0) for word in tokenizar(sentence)]

In [ ]:
def decode_sentence(sentence, most_frequent_words):
  words = [most_frequent_words[code] for code in sentence]
  return ' '.join(words)

In [ ]:
# Teste do encode/decode
frase = paragrafos[10]

frase_encodada = encode_sentence(frase, vocab)
frase_reconstruida = decode_sentence(frase_encodada, most_frequent_words)

print('Original:')
print(frase)
print('Encodada:')
print(frase_encodada)
print('Reconstruída:')
print(frase_reconstruida)
print("--------------------------------------")

frase = "pery disse que amava cecilia"

frase_encodada = encode_sentence(frase, vocab)
frase_reconstruida = decode_sentence(frase_encodada, most_frequent_words)

print('Original:')
print(frase)
print('Encodada:')
print(frase_encodada)
print('Reconstruída:')
print(frase_reconstruida)


Original:
Esta nova edição devia dar satisfação do empenho, que a extrema benevolencia do publico ledor, tão minguado ainda, mudou em bem para divida de reconhecimento.
Encodada:
[120, 1157, 2435, 172, 229, 916, 12, 0, 1, 5, 4, 725, 0, 12, 0, 0, 1, 107, 0, 69, 1, 0, 22, 136, 19, 2439, 7, 917, 3]
Reconstruída:
esta nova edição devia dar satisfação do <unk> , que a extrema <unk> do <unk> <unk> , tão <unk> ainda , <unk> em bem para divida de reconhecimento .
--------------------------------------
Original:
pery disse que amava cecilia
Encodada:
[23, 50, 5, 546, 35]
Reconstruída:
pery disse que amava cecilia


## Classe do dataset

Para cada parágrafo, é necessário gerar os dados de treinamento. Supondo que a frase é "eu gosto de pizza." e vamos usar uma janela de contexto igual a 2, a ideia é que essa frase gere o seguinte conjunto de treinamento:

input -> target

[UNK, "eu"] -> "gosto" (ESSE CASO NÃO SERÁ CONSIDERADO POR ENQUANTO)

["eu", "gosto"] -> "de"

["gosto", "de"] -> "pizza"

["de", "pizza"] -> "."

In [ ]:
def gera_inputs_e_targets_para_array(array, n):
  # Faz uma janela deslizante de tamanho n no array
  janelas = []
  targets = []

  for i in range(len(array) - n):
    janela = array[i:i+n]
    janelas.append(janela)
    targets.append(array[i+n])

  return janelas, targets

In [ ]:
# Exemplo de uso da função gera_inputs_e_targets_para_array
exemplo = "eu gosto de pizza .".split()

for n in range(1, 4):
  print(f'Testando para janela de tamanho {n}')
  inputs, targets = gera_inputs_e_targets_para_array(exemplo, n)

  # Testa
  for input_target in zip(inputs, targets):
    print(f'{input_target[0]} -> {input_target[1]}')
  print('------------------------------')

Testando para janela de tamanho 1
['eu'] -> gosto
['gosto'] -> de
['de'] -> pizza
['pizza'] -> .
------------------------------
Testando para janela de tamanho 2
['eu', 'gosto'] -> de
['gosto', 'de'] -> pizza
['de', 'pizza'] -> .
------------------------------
Testando para janela de tamanho 3
['eu', 'gosto', 'de'] -> pizza
['gosto', 'de', 'pizza'] -> .
------------------------------


In [ ]:
# Testa com um parágrafo real e o tamanho do contexto configurado
i = 60
inputs, targets = gera_inputs_e_targets_para_array(tokenizar(paragrafos[i]), context_size)
print(paragrafos[i])
for input_target in zip(inputs, targets):
  print(f'{input_target[0]} -> {input_target[1]}')

Descobrindo-se, curvou o joelho em terra, e estendeo a mão direita sobre o abysmo, cujos échos adormecidos repetirão ao longe a ultima phrase do juramento prestado sobre o altar da natureza, em face do sol que transmontava.
['descobrindo', '-', 'se', ',', 'curvou', 'o', 'joelho', 'em', 'terra'] -> ,
['-', 'se', ',', 'curvou', 'o', 'joelho', 'em', 'terra', ','] -> e
['se', ',', 'curvou', 'o', 'joelho', 'em', 'terra', ',', 'e'] -> estendeo
[',', 'curvou', 'o', 'joelho', 'em', 'terra', ',', 'e', 'estendeo'] -> a
['curvou', 'o', 'joelho', 'em', 'terra', ',', 'e', 'estendeo', 'a'] -> mão
['o', 'joelho', 'em', 'terra', ',', 'e', 'estendeo', 'a', 'mão'] -> direita
['joelho', 'em', 'terra', ',', 'e', 'estendeo', 'a', 'mão', 'direita'] -> sobre
['em', 'terra', ',', 'e', 'estendeo', 'a', 'mão', 'direita', 'sobre'] -> o
['terra', ',', 'e', 'estendeo', 'a', 'mão', 'direita', 'sobre', 'o'] -> abysmo
[',', 'e', 'estendeo', 'a', 'mão', 'direita', 'sobre', 'o', 'abysmo'] -> ,
['e', 'estendeo', 'a', 'm

In [ ]:
# Testa com um parágrafo real, mas agora usa ele encodado em vez de usar as palavras do texto
inputs, targets = gera_inputs_e_targets_para_array(encode_sentence(paragrafos[i], vocab), context_size)
print(paragrafos[i])
for input_target in zip(inputs, targets):
  print(f'{input_target[0]} -> {input_target[1]}')

Descobrindo-se, curvou o joelho em terra, e estendeo a mão direita sobre o abysmo, cujos échos adormecidos repetirão ao longe a ultima phrase do juramento prestado sobre o altar da natureza, em face do sol que transmontava.
[0, 2, 9, 1, 1413, 6, 1414, 22, 131] -> 1
[2, 9, 1, 1413, 6, 1414, 22, 131, 1] -> 8
[9, 1, 1413, 6, 1414, 22, 131, 1, 8] -> 0
[1, 1413, 6, 1414, 22, 131, 1, 8, 0] -> 4
[1413, 6, 1414, 22, 131, 1, 8, 0, 4] -> 94
[6, 1414, 22, 131, 1, 8, 0, 4, 94] -> 820
[1414, 22, 131, 1, 8, 0, 4, 94, 820] -> 39
[22, 131, 1, 8, 0, 4, 94, 820, 39] -> 6
[131, 1, 8, 0, 4, 94, 820, 39, 6] -> 733
[1, 8, 0, 4, 94, 820, 39, 6, 733] -> 1
[8, 0, 4, 94, 820, 39, 6, 733, 1] -> 1587
[0, 4, 94, 820, 39, 6, 733, 1, 1587] -> 2072
[4, 94, 820, 39, 6, 733, 1, 1587, 2072] -> 2488
[94, 820, 39, 6, 733, 1, 1587, 2072, 2488] -> 0
[820, 39, 6, 733, 1, 1587, 2072, 2488, 0] -> 28
[39, 6, 733, 1, 1587, 2072, 2488, 0, 28] -> 205
[6, 733, 1, 1587, 2072, 2488, 0, 28, 205] -> 4
[733, 1, 1587, 2072, 2488, 0, 28, 

In [ ]:
%%time
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

class ParagrafosDataset(Dataset):
  def __init__(self, paragrafos, vocab, context_size):
    # Salva o vocabulário
    self.vocab = vocab
    # Cria os inputs e target
    inputs = []
    targets = []

    for p in paragrafos:
      # O primeiro passo é pegar cada frase do parágrafo e encodar
      p_tokenizado = encode_sentence(p, self.vocab)
      # Só faz sentido considerar frases que tem no mínimo (context_size + 1) tokens
      if (len(p_tokenizado) <= context_size):
        continue

      # Agora vamos gerar os dados de treinamento para esse parágrafo
      p_inputs, p_targets = gera_inputs_e_targets_para_array(p_tokenizado, context_size)

      # Adiciona independentemente se tiver UKN ou não no input ou target
      inputs.extend(p_inputs)
      targets.extend(p_targets)

      # Apenas adiciona se o input ou o target não tiver nenhum UNK (código 0)
      #for p_um_input, p_um_target in zip(p_inputs, p_targets):
      #  if (0 not in p_um_input and p_um_target != 0):
      #    inputs.append(p_um_input)
      #    targets.append(p_um_target)

    # Mantém em cache
    self.inputs = torch.tensor(inputs)
    self.targets = torch.tensor(targets)

  def __len__(self):
    return len(self.targets)

  def __getitem__(self, idx):
    return self.inputs[idx], self.targets[idx]

CPU times: user 1.89 s, sys: 376 ms, total: 2.27 s
Wall time: 6.86 s


In [ ]:
# Testa a implementação do Dataset

teste_paragrafos = ["Depois, vendo que esta expedição não se realisava, e que seu braço e sua coragem de nada valião ao rei de Portugal", "Colocando uma segunda frase qualquer apenas para ver se está tudo certo."]
teste_dataset = ParagrafosDataset(teste_paragrafos, vocab, context_size)

print('Imprimindo o dataset')
for dados in teste_dataset:
  print(dados)

print('-------------------------')
print('Como deveria estar (testando se o dataset está considerando corretamente os parágrafos:')
for p in teste_paragrafos:
  # Faz o encode do parágrafo
  p_encodado = encode_sentence(p, vocab)
  inputs, targets = gera_inputs_e_targets_para_array(p_encodado, context_size)
  for inputs_targets in zip(inputs, targets):
    print(torch.tensor(inputs_targets[0]), torch.tensor(inputs_targets[1]))

Imprimindo o dataset
(tensor([ 63,   1, 275,   5, 120, 994,  13,   9,   0]), tensor(1))
(tensor([  1, 275,   5, 120, 994,  13,   9,   0,   1]), tensor(8))
(tensor([275,   5, 120, 994,  13,   9,   0,   1,   8]), tensor(5))
(tensor([  5, 120, 994,  13,   9,   0,   1,   8,   5]), tensor(20))
(tensor([120, 994,  13,   9,   0,   1,   8,   5,  20]), tensor(204))
(tensor([994,  13,   9,   0,   1,   8,   5,  20, 204]), tensor(8))
(tensor([ 13,   9,   0,   1,   8,   5,  20, 204,   8]), tensor(18))
(tensor([  9,   0,   1,   8,   5,  20, 204,   8,  18]), tensor(363))
(tensor([  0,   1,   8,   5,  20, 204,   8,  18, 363]), tensor(7))
(tensor([  1,   8,   5,  20, 204,   8,  18, 363,   7]), tensor(252))
(tensor([  8,   5,  20, 204,   8,  18, 363,   7, 252]), tensor(0))
(tensor([  5,  20, 204,   8,  18, 363,   7, 252,   0]), tensor(28))
(tensor([ 20, 204,   8,  18, 363,   7, 252,   0,  28]), tensor(550))
(tensor([204,   8,  18, 363,   7, 252,   0,  28, 550]), tensor(7))
(tensor([  8,  18, 363,   7, 2

### Instâncias vocabulário de treino, dataset e dataloader

Gera datasets de treinamento e de teste:

- Vou fazer a consideração de que a proporção é no total de parágrafos, e não no total do conjunto de dados. Como cada parágrafo tem um total de frases/palavras diferentes, o conjunto final não ficará com a proporção exatamente conforme esperado inicialmente. Entretanto, pensando que em um texto as coisas são mais ou menos distribuídas, espera-se que, no final, a proporção seja mais ou menos conforme a desejada.

- Depois de fazer isso, é necessário gerar novamente o vocabulário, mas considerando apenas o conjunto de treinamento.

In [ ]:
from sklearn.model_selection import train_test_split

train_paragrafos, val_paragrafos = train_test_split(paragrafos, test_size=test_size, random_state=seed)

In [ ]:
# Gera novamente o vocabulário, mas agora usando apenas os parágrafos de treinamento.
# Isso é necessário pois o treino deveria levar em consideração apenas o vocabulário de treinamento
# Num exemplo real, com bases de dados grandes, é improvável que a base de validação tenha
# palavras que não estão na base de treinamento.
# Entretanto, no exercício, para bases pequenas, é comum de ocorrer.
vocab_size, vocab, most_frequent_words = gerar_vocabulario(train_paragrafos, vocab_size_desejado_sem_UNK)
print(vocab_size)

3001


In [ ]:
# Gera os dataset de treino e validação
train_data = ParagrafosDataset(train_paragrafos, vocab, context_size)
val_data = ParagrafosDataset(val_paragrafos, vocab, context_size)

In [ ]:
print(f'len(val_data): {len(val_data)}')
print(f'len(train_data): {len(train_data)}')
print(f'Proporção de teste (deve ser próximo de {test_size}): {len(val_data)/(len(train_data)+len(val_data))}')

len(val_data): 18881
len(train_data): 76116
Proporção de teste (deve ser próximo de 0.2): 0.1987536448519427


In [ ]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

## Model

### Testes de dimensões do que se espera do modelo

In [ ]:
import math
import torch.nn as nn
import numpy as np

# Cria um dataset com poucos dados e pega um batch
dataset_pequeno = ParagrafosDataset(paragrafos[0:10], vocab, context_size)
loader_pequeno = DataLoader(dataset_pequeno, batch_size=2, shuffle=False)
inputs, targets = next(iter(loader_pequeno))

print('O batch tem tamanho: ', inputs.shape)
print(f'São {targets.shape} amostras e {inputs.shape[1]} entradas (tamando do contexto)')

# Gera os embeddings
print(f'Cria um nn.Embedding de tamanho ({vocab_size}, {m})')
C = nn.Embedding(vocab_size, m)
print(C, ' => (tamanho do vocabulário, dimensão dos embeddings) \n')

# Passa o batch pela matriz C
print('x = C(inputs)')
x = C(inputs)
print(f'Dimensões de x: {x.shape}')
print(f'Tamanho do batch:\t\tB = {x.shape[0]}')
print(f'Tamanho da sequência:\t\tL = {x.shape[1]}')
print(f'Dimensão dos embeddings:\tD = {x.shape[2]}\n')

# Cria três matrizes WQ, WK e WV de dimensão
print(f'Cria matrizes WQ, WK, WV de tamanhos ({m}, {m}) -> dimensões dos embeddings')
WQ = nn.Parameter(torch.randn(m, m))
WK = nn.Parameter(torch.randn(m, m))
WV = nn.Parameter(torch.randn(m, m))
print(f'WQ = {WQ.shape}')
print(f'WK = {WK.shape}')
print(f'WV = {WV.shape}\n')

# Aplica as matrizes para criar Q, K, V
print(f'Aplica as matrizes WQ, WK e WV em x para criar Q, K e V')
Q = torch.matmul(x, WQ)
K = torch.matmul(x, WK)
V = torch.matmul(x, WV)
print(f'Q = {Q.shape}')
print(f'K = {K.shape}')
print(f'V = {V.shape}\n')

# MATMUL (Q, K) E SCALE
print('Calcula matmul(Q, K.t())')
print('As dimensões de Q e K são (batch, context_size, embeddings)')
print('As dimensões de Q e K.t() são (batch, context_size, embeddings) x (batch, embeddings, context_size)')
print('As dimensões finais devem ser (batch, context_size, context_size)')
qkt = torch.bmm(Q, K.transpose(1, 2)) / math.sqrt(m)
print(f'qkt = {qkt.shape}\n')

# SOFTMAX
print('Aplica o softmax em qkt. Dimensões permanecem')
qkt_softmax = torch.softmax(qkt, dim=2)
print(f'qkt_softmax = {qkt_softmax.shape}\n')

# MATMUL (_, V)
print('Calcula o QKV')
print(f'As dimensões de qkt_softmax são (batch, context_size, context_size)')
print(f'As dimensões de v são (batch, context_size, embeddings)')
print(f'As dimensões resultantes devem ser (batch, context_size, embeddings)')
qkv = torch.bmm(qkt_softmax, V)
print(f'qkv = {qkv.shape}\n')

# Daqui pra frente, segue a lógica normal.
# Esse qkv passa a ser a nova entrada
# Diferentemente do exercício anterior, vou considerar, para facilitar, que a entrada será sempre em batch
print('Achata a entrada:')
x = qkv
batch_size, _, _ = x.shape
x = x.view(batch_size, -1)
print(f'x = {x.shape}')

# Cria a primeira camada
print('Cria a primeira camada: d_plus_H')
d_plus_H = nn.Linear(in_features=context_size*m, out_features=h, bias=True)
o = d_plus_H(x)
print(f'Primeira camada: {o.shape}\n')

# Passa pela ativação
print('Passa pela ativação')
o = nn.ReLU()(o)
print(f'Ativação: {o.shape}\n')

# Última camada
print('Última camada')
b_plus_U = nn.Linear(in_features=h, out_features=vocab_size, bias=True)
o = b_plus_U(o)
print(f'Última camada: {o.shape}\n')

O batch tem tamanho:  torch.Size([2, 9])
São torch.Size([2]) amostras e 9 entradas (tamando do contexto)
Cria um nn.Embedding de tamanho (3001, 64)
Embedding(3001, 64)  => (tamanho do vocabulário, dimensão dos embeddings) 

x = C(inputs)
Dimensões de x: torch.Size([2, 9, 64])
Tamanho do batch:		B = 2
Tamanho da sequência:		L = 9
Dimensão dos embeddings:	D = 64

Cria matrizes WQ, WK, WV de tamanhos (64, 64) -> dimensões dos embeddings
WQ = torch.Size([64, 64])
WK = torch.Size([64, 64])
WV = torch.Size([64, 64])

Aplica as matrizes WQ, WK e WV em x para criar Q, K e V
Q = torch.Size([2, 9, 64])
K = torch.Size([2, 9, 64])
V = torch.Size([2, 9, 64])

Calcula matmul(Q, K.t())
As dimensões de Q e K são (batch, context_size, embeddings)
As dimensões de Q e K.t() são (batch, context_size, embeddings) x (batch, embeddings, context_size)
As dimensões finais devem ser (batch, context_size, context_size)
qkt = torch.Size([2, 9, 9])

Aplica o softmax em qkt. Dimensões permanecem
qkt_softmax = torch

### Testes de implementação em laço vs matricial

In [ ]:
%%time
# Já temos disponível:
#   - Matrizes C, WQ, WK e WV
#   - Batch
#   - Resultados intermediários:
#       - Q = x * WQ
#       - K = x * WK
#       - V = x * WV
#       - Q x K_T / sqrt(m)
#       - [Q x K_T / sqrt(m)] x V <- mecanismo de atenção


############################################################
# A ideia desse teste é validar a implementação matricial,
# ver se ela está batendo com uma implementação em laço
# (mais ineficiente, porém mais simples de entender).
# Por isso, faremos passo a passo aqui, sem preocupar em
# otimizar os laços. A ideia é comparar os passos intermediários
############################################################

# Passa o batch pela matriz C
# x tem tamanho (B, context_size, m)
x = C(inputs)

# O tamanho do batch (B) não é recebido como parâmetro, então vamos pegar ele aqui:
B, _, _ = x.shape

# Implementa a operação matricial x * WQ, x * WK, x * WV
# WQ, WK e WX tem tamanho (m, m)
# A saída dessa operação tem que ter tamanho (B, context_size, m) * (m, m) = (B, context_size, m)
# A saída dessa operação tem que ter o tamanho
if comparacoes_loop['projecoes']:
  Q_laco = torch.zeros(B, context_size, m)
  K_laco = torch.zeros(B, context_size, m)
  V_laco = torch.zeros(B, context_size, m)

  for b in range(B):
    for i in range(context_size):
      for j in range(m):
        for aux in range(m):
          Q_laco[b, i, j] += x[b, i, aux] * WQ[aux, j]
          K_laco[b, i, j] += x[b, i, aux] * WK[aux, j]
          V_laco[b, i, j] += x[b, i, aux] * WV[aux, j]

  # Considerando precisão de 1e-6. Não dá precisão maior do que isso...
  print('Testando cálculo das projeções usando for loops:')
  print('Q == Q_laco:', torch.allclose(Q, Q_laco, atol=1e-6))
  print('K == K_laco:', torch.allclose(K, K_laco, atol=1e-6))
  print('V == V_laco:', torch.allclose(V, V_laco, atol=1e-6), '\n')

if comparacoes_loop['qkt']:
  qkt_laco = torch.zeros(B, context_size, context_size)

  for b in range(B):
    for i in range(context_size):
      for j in range(context_size):
        for aux in range(m):
          qkt_laco[b, i, j] += Q[b, i, aux] * K[b, j, aux]
  qkt_laco = qkt_laco / math.sqrt(m)
  print('Testando cálculo de Q * K.t() usando for loops:')
  print('qkt == qkt_laco:', torch.allclose(qkt, qkt_laco, atol=1e-6), '\n')

if comparacoes_loop['softmax']:
  qkt_softmax_laco = torch.zeros(B, context_size, context_size)

  for b in range(B):
    for i in range(context_size):
      qkt_softmax_laco[b, i, :] = torch.softmax(qkt[b, i, :], dim=0)
  print('Testando cálculo do softmax usando for loops:')
  print('qkt_softmax == qkt_softmax_laco:', torch.allclose(qkt_softmax, qkt_softmax_laco, atol=1e-6), '\n')

if comparacoes_loop['qkv']:
  qkv_laco = torch.zeros(B, context_size, m)

  for b in range(B):
    for i in range(context_size):
      for j in range(m):
        for aux in range(context_size):
          qkv_laco[b, i, j] += qkt_softmax[b, i, aux] * V[b, aux, j]

  print('Testando cálculo de QKV usando for loops:')
  print('qkv == qkv_laco:', torch.allclose(qkv, qkv_laco, atol=1e-6), '\n')

Testando cálculo de Q * K.t() usando for loops:
qkt == qkt_laco: True 

Testando cálculo do softmax usando for loops:
qkt_softmax == qkt_softmax_laco: True 

Testando cálculo de QKV usando for loops:
qkv == qkv_laco: True 

CPU times: user 1.82 s, sys: 231 ms, total: 2.05 s
Wall time: 3.38 s


### Criação de um layer para implementar o mecanismo de atenção

In [ ]:
class AutoAtencao(torch.nn.Module):
  def __init__(self, m):
    super(AutoAtencao, self).__init__()
    self.WQ = nn.Parameter(torch.randn(m, m))
    self.WK = nn.Parameter(torch.randn(m, m))
    self.WV = nn.Parameter(torch.randn(m, m))
    torch.nn.init.xavier_uniform_(self.WQ)
    torch.nn.init.xavier_uniform_(self.WK)
    torch.nn.init.xavier_uniform_(self.WV)

  def forward(self, x):
    Q = torch.matmul(x, self.WQ)
    K = torch.matmul(x, self.WK)
    V = torch.matmul(x, self.WV)

    qkt = torch.bmm(Q, K.transpose(1, 2)) / math.sqrt(m)
    qkt_softmax = torch.softmax(qkt, dim=2)
    qkv = torch.bmm(qkt_softmax, V)

    return qkv

### Encoding de posição

Baseado no código disponível no [tutorial de Transformers do PyTorch](https://pytorch.org/tutorials/beginner/transformer_tutorial.html), mas adaptado para usar a equação do artigo [Attention is all you need](https://arxiv.org/pdf/1706.03762.pdf).

In [ ]:
class EncodingPosicao(torch.nn.Module):
  def __init__(self, context_size, m, device):
    super(EncodingPosicao, self).__init__()

    # Os encodings posicionais serão somados à entrada. Devem ter, portanto,
    # o mesmo tamanho da entrada. A entrada desse layer é a saída de um
    # self.C (os embeddings). Logo, tem tamanho (B, context_size, m).
    # Mas como a matriz é única por batch, podemos criar apenas como
    # (context_size, m) e, depois, fazer o broadcasting na primeira dimensão
    self.pe = torch.zeros(context_size, m)

    position = torch.arange(context_size).unsqueeze(1)
    dois_i = torch.arange(0, m, 2) # = [0, 2, 4, 6, ...., 62]
    div_term = torch.pow(10000, dois_i / m)

    # pe(pos, 2i) -> índices pares
    self.pe[:, 0::2] = torch.sin(position * div_term)

    # pe(pos, 2i+1) -> índices ímpares
    # ESSE CÓDIGO SÓ VAI FUNCIONAR SE m FOR PAR
    self.pe[:, 1::2] = torch.cos(position * div_term)

    self.pe = self.pe.to(device)

  def forward(self, x):
    # Faz o broadcasting de self.pe (context_size, m) para (B, context_size, m)
    return x + self.pe

### LanguageModel

In [ ]:
import torch.nn as nn

class LanguageModel(torch.nn.Module):
  def __init__(self, vocab_size, context_size, m, h, device, ativacao='relu', zerar_w=True):
    super(LanguageModel, self).__init__()

    self.C = nn.Embedding(vocab_size, m)
    self.encoding_posicao = EncodingPosicao(context_size, m, device)
    self.auto_atencao = AutoAtencao(m)
    self.d_plus_H = nn.Linear(in_features=context_size*m, out_features=h, bias=True)
    self.ativacao = nn.ReLU() if ativacao == 'relu' else nn.Tanh()
    self.b_plus_U = nn.Linear(in_features=h, out_features=vocab_size, bias=True)

    self.zerar_w = zerar_w
    # Modelo do artigo:
    if not self.zerar_w:
      self.W = nn.Linear(in_features=context_size*m, out_features=vocab_size, bias=False)

  def forward(self, w):
    # w é uma entrada de tamanho context_size (no artigo é chamada de n)
    # O primeiro passo é obter os embeddings de w
    x = self.C(w)

    # Insere encoding de posição
    x = self.encoding_posicao(x)

    # Agora passa pelo mecanismo de auto atenção
    x = self.auto_atencao(x)

    # E agora volta para o esquema normal do modelo do Bengio
    # Planifica a entrada nas duas últimas dimensões
    if x.dim() == 3: # Usando batchs
      batch_size, _, _ = x.shape
      x = x.view(batch_size, -1)
    elif x.dim() == 2: # Calculando sem usar batch, usando um tensor direto
      x = x.view(-1)

    # Passa pela primeira camada
    o = self.d_plus_H(x)

    # Camada de ativação
    o = self.ativacao(o)

    # Segunda camada
    o = self.b_plus_U(o)

    if not self.zerar_w: # Modelo do artigo
      o = o + self.W(x)

    return o

## Training

In [ ]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
  print('GPU:', torch.cuda.get_device_name(torch.cuda.current_device()))
else:
  print('using CPU')

GPU: Tesla T4


### Função para calcular a loss e a perplexidade para um DataLoader de entrada

In [ ]:
import math
from tqdm import tqdm

def calcula_loss_e_perplexidade(model, loader):
  criterion = nn.CrossEntropyLoss(reduction='sum')
  with torch.no_grad(): # Garante que nenhum gradiente seja calculado
    model.eval()  # Coloca o modelo no modo de avaliação (não treinamento)
    loss = 0.0
    acc = 0
    for inputs, targets in tqdm(loader, desc='Calculando loss e perplexidade'):
      inputs = inputs.to(device)
      targets = targets.to(device)
      # Forward pass
      outputs = model(inputs)
      # Acumula a perda
      loss += criterion(outputs, targets)
      acc += len(targets)

    loss = loss/acc
    ppl = math.exp(loss)

    return loss, ppl

In [ ]:
def print_loss_ppl(msg, loss, ppl):
  print(f'{msg}. Loss: {loss:.2f}. Perplexidade: {ppl:.2f}\n')

In [ ]:
# Model instantiation
model = LanguageModel(vocab_size, context_size, m, 10, device)
model.to(device)

dataset_pequeno = ParagrafosDataset(paragrafos[0:10], vocab, context_size)
loader_pequeno = DataLoader(dataset_pequeno, batch_size=2, shuffle=False)

loss, ppl = calcula_loss_e_perplexidade(model, loader_pequeno)
print_loss_ppl('\nAntes de iniciar o treinamento', loss, ppl)

Calculando loss e perplexidade: 100%|██████████| 12/12 [00:02<00:00,  5.37it/s]


Antes de iniciar o treinamento. Loss: 8.09. Perplexidade: 3249.09



### Função para treinar um modelo

In [ ]:
import torch.optim as optim

def treina_modelo(model, optimizer, train_loader, val_loader, num_epochs=num_epochs):
  print(f'------------------ ANTES DE INICIAR O TREINAMENTO ------------------')
  loss, ppl = calcula_loss_e_perplexidade(model, train_loader)
  print_loss_ppl(f'[TRAIN]', loss, ppl)

  loss, ppl = calcula_loss_e_perplexidade(model, val_loader)
  print_loss_ppl(f'[EVAL]', loss, ppl)

  criterion = nn.CrossEntropyLoss(reduction='mean')
  model.to(device)

  for epoch in range(num_epochs):
    model.train()

    print(f'------------------ [ÉPOCA {epoch+1}/{num_epochs}] ------------------')
    estimativa_loss_epoca_i = 0
    acc_dados = 0

    for inputs, targets in tqdm(train_loader, desc='Treinando modelo'):
      # Envia inputs e target para o mesmo dispositivo
      inputs = inputs.to(device)
      targets = targets.to(device)

      # Forward pass
      outputs = model(inputs)

      # Calcula loss no batch
      loss = criterion(outputs, targets)

      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # Acumula a loss pra época atual
      # Obs.: Isso é só uma estimativa para a loss de treino após a época i.
      # Como os pesos são atualizados após rodar cada batch, ao final da época
      # é esperado que a loss no conjunto de treinamento seja menor
      # do que o calculado dessa forma (o ajuste em cada batch tende a ir
      # convergindo e, consequentemente, diminuindo a loss)
      estimativa_loss_epoca_i += loss.item() * len(train_loader)
      acc_dados += len(train_loader)

    estimativa_loss_epoca_i = estimativa_loss_epoca_i / acc_dados

    # Imprime a estimativa da loss/ppl de treino e a loss no dataloader de avaliação:
    print_loss_ppl(f'[TRAIN ESTIMATIVA]', estimativa_loss_epoca_i, math.exp(estimativa_loss_epoca_i))
    loss, ppl = calcula_loss_e_perplexidade(model, val_loader)
    print_loss_ppl(f'[EVAL]', loss, ppl)

    # Salva os modelos intermediários
    checkpoint_path = f"modelo_epoca_{epoch+1}.pth"
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, checkpoint_path)

### Função para escrever uma frase (recebe um modelo como entrada)

In [ ]:
%%time

# Cria um dataset sem target para guardar apenas uma frase
class UmaFraseDataset(Dataset):
  def __init__(self, frase, vocab, context_size):
    # Salva o vocabulário
    self.vocab = vocab
    # Cria os inputs
    frase_encodada = encode_sentence(frase, self.vocab)
    self.inputs = torch.tensor(frase_encodada[len(frase_encodada)-context_size:len(frase_encodada)])

  def __len__(self):
    return 1

  def __getitem__(self, idx):
    return self.inputs

CPU times: user 38 µs, sys: 7 µs, total: 45 µs
Wall time: 48.4 µs


In [ ]:
def escrever_frase(modelo, vocab, most_frequent_words, entrada, context_size, n_proximas_palavras, descartar_ukn=True):
  if (n_proximas_palavras == 0):
    return entrada
  else:
    # Cria um dataloader com esse input
    loader_uma_frase = DataLoader(UmaFraseDataset(entrada, vocab, context_size), batch_size=1, shuffle=False)
    batch_uma_frase = next(iter(loader_uma_frase)).to(device)

    # Pega só as context_size últimas
    with torch.no_grad():
      output = model(batch_uma_frase)
      softmax = nn.functional.softmax(output, dim=1)

      if descartar_ukn:
        valores, indices = softmax.topk(2, dim=1)
        melhor_not_ukn = indices[0][0].item() if indices[0][0].item() != 0 else indices[0][1].item()
        predicao = most_frequent_words[melhor_not_ukn]
      else:
        argmax = softmax.argmax(dim=0)
        predicao = most_frequent_words[argmax[0]]

    # Substitui símbolos que foram trocados manualmente
    predicao = predicao.replace('SUBSTITUIRPORTRESPONTOS', '...')

  return escrever_frase(modelo, vocab, most_frequent_words, f'{entrada} {predicao}', context_size, n_proximas_palavras-1)

### Inicializa um modelo e vê que frase ele gera (sem treinar):

In [ ]:
# Como esse é o modelo que será treinado, seta a seed aqui
torch.manual_seed(seed)

model = LanguageModel(vocab_size, context_size, m, h, device, ativacao, zerar_w)
model.to(device)

# Escreve uma frase com o modelo sem estar treinado
frase = "O espectaculo que se ofereceu aos seus olhos causou"
print(escrever_frase(model, vocab, most_frequent_words, frase, context_size, 10))

O espectaculo que se ofereceu aos seus olhos causou áquelles cascata ironia pallidos áquelles dedicação pallidos graciosas guerra era


### Treina o modelo

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
treina_modelo(model, optimizer, train_loader, val_loader, num_epochs=num_epochs)

------------------ ANTES DE INICIAR O TREINAMENTO ------------------


Calculando loss e perplexidade: 100%|██████████| 595/595 [00:01<00:00, 422.00it/s]


[TRAIN]. Loss: 8.03. Perplexidade: 3078.86



Calculando loss e perplexidade: 100%|██████████| 148/148 [00:00<00:00, 542.76it/s]


[EVAL]. Loss: 8.03. Perplexidade: 3076.87

------------------ [ÉPOCA 1/10] ------------------


...

[EVAL]. Loss: 4.64. Perplexidade: 103.65

------------------ [ÉPOCA 10/10] ------------------


Treinando modelo: 100%|██████████| 595/595 [00:01<00:00, 356.50it/s]


[TRAIN ESTIMATIVA]. Loss: 4.28. Perplexidade: 72.60



Calculando loss e perplexidade: 100%|██████████| 148/148 [00:00<00:00, 731.29it/s]

[EVAL]. Loss: 4.64. Perplexidade: 103.15



## Avaliação

In [ ]:
def recupera_modelo(model, epoca):
  # Recupera o modelo salvo na época x
  checkpoint_path = f"modelo_epoca_{epoca}.pth"
  # Carregar o estado do checkpoint
  checkpoint = torch.load(checkpoint_path)
  # Aplicar o estado do modelo e otimizador carregados
  model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
# Completa frases
def completa_frase_do_conjunto(context_size, paragrafos, idx_para_completar, epoca_modelo_salvo, descartar_ukn=False):
  for i in idx_para_completar:
    frase_esperada = paragrafos[i]
    palavras_na_frase = frase_esperada.split(' ')
    palavras_na_frase = palavras_na_frase[0:context_size]

    if len(palavras_na_frase) == context_size:
      frase = ' '.join(palavras_na_frase)
      recupera_modelo(model, epoca_modelo_salvo)
      print('-----------------------------------------------------------------')
      print(f'Testando para o índice {i}')
      print(f'Modelo da epoca {epoca_modelo_salvo}:')
      print('Início:  ', frase)
      print('Correta: ', frase_esperada)
      print('Gerada:  ', escrever_frase(model, vocab, most_frequent_words, frase, context_size, 30, descartar_ukn=descartar_ukn))

### Avaliação no conjunto de treinamento

In [ ]:
# Ver como ficou NO CONJUNTO DE TREINAMENTO depois de rodar por muitas épocas (overfit no modelo)
completa_frase_do_conjunto(context_size, train_paragrafos, [0, 1, 2, 3, 55, 61, 76, 78, 388, 555, 1000], epoca_modelo_salvo=num_epochs, descartar_ukn=True)

-----------------------------------------------------------------
Testando para o índice 0
Modelo da epoca 10:
Início:   --Não; estou tão bem aqui! Não foste tu que
Correta:  --Não; estou tão bem aqui! Não foste tu que me trouxeste?
Gerada:   --Não; estou tão bem aqui! Não foste tu que não ha pouco tendes , e a que o indio que o indio de sua senhora , que não se tivesse de sua senhora . não vos se não te
-----------------------------------------------------------------
Testando para o índice 1
Modelo da epoca 10:
Início:   --Como vos illudis! Quem o julgará criminoso? Vós? Pois
Correta:  --Como vos illudis! Quem o julgará criminoso? Vós? Pois bem; outros o julgarão innocente e o defenderão; e não tereis remedio senão curvar a cabeça e calar-vos.
Gerada:   --Como vos illudis! Quem o julgará criminoso? Vós? Pois que o indio que o indio que o indio que o indio que o indio que o indio que o indio que o indio que o indio que o indio
----------------------------------------------------------

### Avaliação no conjunto de teste

In [ ]:
epoca_melhor_modelo = 7

# Ver como ficou NO CONJUNTO DE TREINAMENTO
completa_frase_do_conjunto(context_size, val_paragrafos, range(100,110), epoca_modelo_salvo=epoca_melhor_modelo, descartar_ukn=True)

-----------------------------------------------------------------
Testando para o índice 100
Modelo da epoca 7:
Início:   --Aqui me tendes! repetio o cavalheiro. Dizei o que
Correta:  --Aqui me tendes! repetio o cavalheiro. Dizei o que quereis de D. Antonio de Mariz, e dizei-o claro e breve. Se fôr de justiça, sereis satisfeitos; se fôr uma falta, tereis a punição que merecerdes.
Gerada:   --Aqui me tendes! repetio o cavalheiro. Dizei o que se não se o seu amor . antonio o seu amor ; o seu amor , e a sua vida , a sua vida , e a sua vida ,
-----------------------------------------------------------------
Testando para o índice 101
Modelo da epoca 7:
Início:   --Então devo tirar o meu; já não estamos irmãs.
Correta:  --Então devo tirar o meu; já não estamos irmãs.
Gerada:   --Então devo tirar o meu; já não estamos irmãs. que o seu amor , não o que o seu amor , e o seu amor ; o italiano que o seu amor , e a sua vida , a
-----------------------------------------------------------------
Tes

## Exemplo de uso

In [ ]:
recupera_modelo(model, epoca=epoca_melhor_modelo)

frase = "Se se tratasse de sua vida, Pery teria sangue"
print(f'Modelo da epoca {epoca_melhor_modelo}:')
print(frase)
print(escrever_frase(model, vocab, most_frequent_words, frase, context_size, 30, descartar_ukn=True))

Modelo da epoca 7:
Se se tratasse de sua vida, Pery teria sangue
Se se tratasse de sua vida, Pery teria sangue o seu amor , e o seu amor ; o italiano que o seu amor , e a sua vida , a sua vida , e a sua vida ,
